<a href="https://colab.research.google.com/github/bessafelipe/Projetos-freeCodeCamp/blob/main/SMS_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*Note: You are currently reading this using Google Colaboratory which is a cloud-hosted version of Jupyter Notebook. This is a document containing both text cells for documentation and runnable code cells. If you are unfamiliar with Jupyter Notebook, watch this 3-minute introduction before starting this challenge: https://www.youtube.com/watch?v=inN8seMm7UI*

---

In this challenge, you need to create a machine learning model that will classify SMS messages as either "ham" or "spam". A "ham" message is a normal message sent by a friend. A "spam" message is an advertisement or a message sent by a company.

You should create a function called `predict_message` that takes a message string as an argument and returns a list. The first element in the list should be a number between zero and one that indicates the likeliness of "ham" (0) or "spam" (1). The second element in the list should be the word "ham" or "spam", depending on which is most likely.

For this challenge, you will use the [SMS Spam Collection dataset](http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/). The dataset has already been grouped into train data and test data.

The first two cells import the libraries and data. The final cell tests your model and function. Add your code in between these cells.


In [ ]:

%tensorflow_version 2.x  # this line is not required unless you are in a notebook
from keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences
import keras
import tensorflow as tf
import os
import numpy as np
import tensorflow as tf
import pandas as pd
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `2.x  # this line is not required unless you are in a notebook`. This will be interpreted as: `2.x`.


TensorFlow 2.x selected.
2.8.0


In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

--2022-05-02 19:11:00--  https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 172.67.70.149, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 358233 (350K) [text/tab-separated-values]
Saving to: ‘train-data.tsv’

train-data.tsv      100%[===================>] 349.84K  --.-KB/s    in 0.04s   

2022-05-02 19:11:01 (9.35 MB/s) - ‘train-data.tsv’ saved [358233/358233]

--2022-05-02 19:11:01--  https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 172.67.70.149, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 118774 (116K) [text/tab-separated-values]
Saving to: ‘valid-data.tsv’

valid-data.tsv      100%[==============

In [14]:
train_df = pd.read_csv(test_file_path, sep="\t", header=None, names=["type", "msg"])
train_df.dropna()
train_df.head()
test_df = pd.read_csv(test_file_path, sep="\t", header=None, names=["type", "msg"])
test_df.dropna()
train_df.head()
train_df["type"] = pd.factorize(train_df["type"])[0]
test_df["type"] = pd.factorize(test_df["type"])[0]

from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words = 1000, oov_token="<OOV>")

train_label=train_df.pop('type')
test_label=test_df.pop('type')

train_label=train_label.to_numpy()
test_label=test_label.to_numpy()

VOCAB_SIZE=88584
MAXLEN = 250
BATCH_SIZE = 64

train_data=train_df.values.tolist()
test_data=test_df.values.tolist()

train_data=[x[0] for x in train_data]
test_data=[x[0] for x in test_data]

tokenizer.fit_on_texts(train_data)
word_index = tokenizer.word_index

train_index = tokenizer.texts_to_sequences(train_data)
test_index = tokenizer.texts_to_sequences(test_data)

train_index = pad_sequences(train_index,MAXLEN)
test_index = pad_sequences(test_index,MAXLEN)



model = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, 32),
    tf.keras.layers.LSTM(32),
    tf.keras.layers.Dense(1, activation="sigmoid")
])
model.compile(loss="binary_crossentropy",optimizer="rmsprop",metrics=['acc'])
history = model.fit(train_index, train_label, epochs=5, validation_data=(test_index,test_label), validation_steps=30)

Epoch 1/5
44/44 [==============================] - 11s 150ms/step - loss: 0.3574 - acc: 0.8649 - val_loss: 0.2221 - val_acc: 0.9073
Epoch 2/5
44/44 [==============================] - 6s 136ms/step - loss: 0.1737 - acc: 0.9382 - val_loss: 0.1232 - val_acc: 0.9727
Epoch 3/5
44/44 [==============================] - 6s 138ms/step - loss: 0.1051 - acc: 0.9756 - val_loss: 0.0760 - val_acc: 0.9799
Epoch 4/5
44/44 [==============================] - 6s 138ms/step - loss: 0.0686 - acc: 0.9828 - val_loss: 0.0570 - val_acc: 0.9820
Epoch 5/5
44/44 [==============================] - 6s 137ms/step - loss: 0.0466 - acc: 0.9892 - val_loss: 0.0467 - val_acc: 0.9849


In [15]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
  pred_text = tokenizer.texts_to_sequences([pred_text])
  pred_text = pad_sequences(pred_text,MAXLEN)
  pre = model.predict([pred_text])
  p = pre[0][0]
  return [p, "ham" if p <0.5 else "spam"]


  return (prediction)

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

[0.0028855503, 'ham']


In [16]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    print(prediction)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()


[0.0028855503, 'ham']
[0.7997376, 'spam']
[0.0027095377, 'ham']
[0.81412685, 'spam']
[0.9428938, 'spam']
[0.0026418865, 'ham']
[0.0055888295, 'ham']
You passed the challenge. Great job!
